In [ ]:
import numpy as np

# Set up matplotlib and use a nicer set of plot parameters
%config InlineBackend.rc = {}
import matplotlib
matplotlib.rc_file("../../templates/matplotlibrc")
import matplotlib.pyplot as plt
%matplotlib inline

The following line is needed to download the example FITS files used in this tutorial.

In [ ]:
from astropy.utils.data import download_file

# Viewing and manipulating data from FITS tables

In [ ]:
from astropy.io import fits

FITS files can often contain large amount of multi-dimensional data and tables.  

In this particular example, I will open a FITS file from a Chandra observation of the Galactic Center.  The file contains a list of events with x and y coordinates, energy, and various other pieces of information.

In [ ]:
event_filename = download_file( 'http://data.astropy.org/tutorials/FITS-tables/chandra_events.fits', cache=True )

## Opening the FITS file and viewing table contents

Since the file is big, I will open with `memmap=True` to prevent RAM storage issues.

In [ ]:
hdu_list = fits.open(event_filename, memmap=True)

In [ ]:
hdu_list.info()

I'm interested in reading EVENTS, which contains information about each X-ray photon that hit the detector.

To find out what information the table contains, I will print the column names.

In [ ]:
print(hdu_list[1].columns)

Now I'll load the data into a separate variable.

In [ ]:
evt_data = hdu_list[1].data

We can extract data from the table by referencing the column name.

For example, I'll make a histogram for the energy of each photon, giving us a sense for the spectrum (folded with the detector's efficiency).

In [ ]:
NBINS = 500
energy_hist = plt.hist(evt_data['energy'], NBINS)

## Making a 2-D histogram with some table data

I will make an image by binning the x and y coordinates of the events into a 2-D histogram.

In [ ]:
x = evt_data['x']
y = evt_data['y']

This particular observation spans five CCD chips.  Here I will pick events that only fell on the main (ACIS-I) chips, which have number ids 0, 1, 2, and 3.

In [ ]:
ii = np.any([evt_data['ccd_id'] == i for i in [0,1,2,3]], axis=0)

### Method 1: Use numpy to make a 2-D histogram and imshow to display it

This method allowed me to create an image without stretching

In [ ]:
NBINS = (100,100)

img_zero, yedges, xedges = np.histogram2d(evt_data['x'][ii], evt_data['y'][ii], NBINS)

extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]

plt.imshow(img_zero, extent=extent, interpolation='nearest', cmap='gist_yarg', origin='lower')

plt.xlabel('x')
plt.ylabel('y')

# To see more color maps
# http://wiki.scipy.org/Cookbook/Matplotlib/Show_colormaps

### Method 2: Use hist2d with a log-normal color scheme

In [ ]:
from matplotlib.colors import LogNorm

In [ ]:
NBINS = (100,100)
img_zero_mpl = plt.hist2d(evt_data['x'][ii], evt_data['y'][ii], NBINS, cmap='viridis', norm=LogNorm())

cbar = plt.colorbar(ticks=[1.0,3.0,6.0])
cbar.ax.set_yticklabels(['1','3','6'])

plt.xlabel('x')
plt.ylabel('y')

## Close the FITS file

In [ ]:
hdu_list.close()

## Exercises

Make a scatter plot of the same data you histogrammed above.  The [plt.scatter](http://matplotlib.org/api/pyplot_api.html#matplotlib.pyplot.scatter) function is your friend for this.  What are the pros and cons of doing this?

Try the same with the [plt.hexbin](http://matplotlib.org/api/pyplot_api.html#matplotlib.pyplot.hexbin) plotting function. Which do you think looks better for this kind of data?